# Energy Data Parser

### Scope for *next* notebook
- outliers
- trend analysis for bad points
- interpolate/resample for missing cells
- ???

In [5]:
# Preliminary setup
import pandas as pd
import zipfile
import os

In [6]:
df = pd.read_pickle('alldata.pickle')

In [7]:
# Show rows with nulls
dfnulls = df[df.isnull().any(axis=1)]
dfnulls

,active,reactive,volts,amps,sub1,sub2,sub3
dtime,,,,,,,
2006-12-21 11:23:00,?,?,?,?,?,?,NaN
2006-12-21 11:24:00,?,?,?,?,?,?,NaN
2006-12-30 10:08:00,?,?,?,?,?,?,NaN
2006-12-30 10:09:00,?,?,?,?,?,?,NaN
2007-01-14 18:36:00,?,?,?,?,?,?,NaN
2007-01-28 17:13:00,?,?,?,?,?,?,NaN
2007-02-22 22:58:00,?,?,?,?,?,?,NaN
2007-02-22 22:59:00,?,?,?,?,?,?,NaN
2007-03-25 17:52:00,?,?,?,?,?,?,NaN


In [13]:
# How many rows have a missing sample?
n_missing = sum(df.isnull().any(axis=1))
n_missing

3724

In [14]:
# What percentage of rows is missing samples?
"{}%".format(round(n_missing / len(df) * 100, 4))

'3.724%'

In [15]:
# How strict is the index, ie. what is the time difference between consecutive elements?
from collections import Counter
deltas = [d for d in map(lambda a: a[1] - a[0], zip(df.index, df.index[1:]))]
Counter(deltas)

Counter({Timedelta('0 days 00:01:00'): 99999})